In [1]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, transform
%matplotlib inline

In [2]:
# S8 to iPad
imnames = ['./Images/Arm(S8+)_resize.jpg', './Images/ATM(S8+)_resize.jpg', './Images/Book(S8+)_resize.jpg', './Images/Bucket(S8+)_resize.jpg', './Images/Field(S8+)_resize.jpg', './Images/Opposite(S8+)_resize.jpg', './Images/Shirt(S8+)_resize.jpg', './Images/Sky(S8+)_resize.jpg', './Images/Snack(S8+)_resize.jpg']

imlist = (io.imread_collection(imnames))

for i in range(len(imlist)):
    m = transform.resize(imlist[i], (32, 1200, 3))

In [3]:
res = np.zeros(shape=(1,3))

for i in range(len(imlist)):
    m = transform.resize(imlist[i], (1200, 1200, 3))
    arr = m.reshape((1200*1200), 3)
    res = np.concatenate((res, arr), axis=0)
    
res = np.delete(res, (0), axis=0)

print(res)
print(len(res))

[[ 0.36797386  0.34052288  0.23856209]
 [ 0.33137255  0.30392157  0.20196078]
 [ 0.34379085  0.31633987  0.21437908]
 ..., 
 [ 0.74901961  0.74117647  0.74509804]
 [ 0.74901961  0.74117647  0.74509804]
 [ 0.74901961  0.74117647  0.74509804]]
12960000


In [4]:
m = res.mean(axis=0)
print(m)

[ 0.5303844   0.49741328  0.45365616]


In [5]:
res = res - m
print(res)

[[-0.16241054 -0.15689041 -0.21509407]
 [-0.19901185 -0.19349171 -0.25169538]
 [-0.18659355 -0.18107341 -0.23927708]
 ..., 
 [ 0.21863521  0.24376319  0.29144188]
 [ 0.21863521  0.24376319  0.29144188]
 [ 0.21863521  0.24376319  0.29144188]]


In [6]:
R = np.cov(res, rowvar=False)
print(R)

[[ 0.06149552  0.05562276  0.04909883]
 [ 0.05562276  0.06570364  0.05940754]
 [ 0.04909883  0.05940754  0.07618774]]


In [7]:
from numpy import linalg as LA
evals, evecs = LA.eigh(R)
idx = np.argsort(evals)[::-1]
evecs = evecs[:, idx]

evals = evals[idx]

evecs = evecs[:, :3]

evecs_mat = np.column_stack((evecs))

m = np.dot(evecs.T, res.T).T

In [8]:
def data_aug(img):
    mu = 0
    sigma = 0.1
    feature_vec=np.matrix(evecs_mat)
    
    se = np.zeros((3,1))
    se[0][0] = np.random.normal(mu, sigma)*evals[0]
    se[1][0] = np.random.normal(mu, sigma)*evals[1]
    se[2][0] = np.random.normal(mu, sigma)*evals[2]
    se = np.matrix(se)
    val = feature_vec*se
    
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            for k in range(img.shape[2]):
                img[i, j, k] = float(img[i, j, k]) + float(val[k])
    #return img

In [9]:
img = imlist[0]
print(img[0])
print("-----------------")
data_aug(img)
print(img[0])
#plt.imshow(img)
print(type(img))

[[95 88 62]
 [88 81 55]
 [81 74 48]
 ..., 
 [21 33 83]
 [21 33 83]
 [21 33 83]]
-----------------
[[95 87 62]
 [88 80 55]
 [81 73 48]
 ..., 
 [21 32 83]
 [21 32 83]
 [21 32 83]]
<class 'numpy.ndarray'>


In [10]:
#import cv2
#cv2.imwrite('arm_pca_jitter.jpg',img)


#from skimage.io import imsave
#imsave('arm(S8)_pca_jittering.jpg',img)

import scipy.misc
scipy.misc.toimage(img, cmin=0.0, cmax=255.0).save('arm(S8)_pca_jittering.jpg')

In [11]:
img = imlist[2]
print(img[0])
print("-----------------")
data_aug(img)
print(img[0])
scipy.misc.toimage(img, cmin=0.0, cmax=255.0).save('Book(S8)_pca_jittering.jpg')

[[173 175 174]
 [173 175 174]
 [170 172 171]
 ..., 
 [195 192 199]
 [196 193 200]
 [197 194 201]]
-----------------
[[173 174 174]
 [173 174 174]
 [170 171 171]
 ..., 
 [195 191 199]
 [196 192 200]
 [197 193 201]]
